In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append("../")

from src.enzyme_hackathon.utils import (one_hot_dict, one_hot_encode, one_hot_encode_screening_data,
                                        one_hot_encode_sequences)
from src.enzyme_hackathon.utils import AA_LABELS
from src.enzyme_hackathon.utils import (create_fc_model, predict, generate_variant, r_squared, nan_mse)
import keras
import tensorflow as tf

from collections import OrderedDict

Using TensorFlow backend.


In [2]:
?tf.save

Object `tf.save` not found.


In [3]:
fc_nn = keras.models.load_model("../models/FCNN_50_ADAM.hfd5", custom_objects={"nan_mse": nan_mse, "r_squared": r_squared})

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


# Load train/test data

In [4]:
regression_df = pd.read_csv("../data/raw/challenge-0.csv")
encoded_split_dict = one_hot_encode_screening_data(regression_df)
x_train, x_test = (encoded_split_dict['x'][0], encoded_split_dict['x'][1])
y_keys = [key for key in encoded_split_dict['y'].keys()]

y_train, y_test = (OrderedDict(), OrderedDict())
for y_key in y_keys:
    y_train[y_key], y_test[y_key] = (encoded_split_dict['y'][y_key]['data'][0], encoded_split_dict['y'][y_key]['data'][1])

# Generate variants

In [5]:
[regression_df.sequence[0]]

['AQSVPWGISRVQAPAAHNRGLTGSGVKVAVLDTGISTHPDLNIRRGGASFVPGEPSTQDGNGHGTHVAGTIAALNNSIGVLGVAPSAELLYAVKVLGASGSSTGSSVSSIAQGLEWAGNNGMTVANLSLGATSPSATLEQAVNAATSRGVLVVAASGNSGAGSISYPARYANAMAVGATDQNNNRASFSQYGAGLDIVAPGVNVQSTYPGSTYASLNGASMATPHVAGAAALVKQKNPSWSNVQIRNHLKNTATSLGGSSTTNNLYGSGLVNAEAATR']

In [6]:
predict(regression_df.sequence[0], fc_nn, encoded_split_dict, one_hot_dict(AA_LABELS))

array([1.4826045 , 0.97867876, 1.2242274 ], dtype=float32)

In [10]:
variants = []
for i in range(10000):
    variant, variant_performance = generate_variant(regression_df.sequence[0], 
                         fc_nn, 
                         encoded_split_dict,
                         AA_LABELS)
    variant_performance['sequence'] = variant
    variants.append(variant_performance)

In [11]:
df_variants = pd.DataFrame(variants) \
    .assign(fitness = lambda x: x['productivity']*x['performance']*x['stability'])

In [12]:
df_variants.plot(kind='hist', bins=100)

In [13]:
df_variants.sort_values('fitness', ascending=False)

,productivity,performance,stability,sequence,fitness
2926,1.414919,1.047267,1.533646,"[A, Q, S, V, P, W, G, I, S, R, V, Q, A, P, A, ...",2.272552
1964,1.843460,1.006274,1.212961,"[A, Q, S, A, P, W, G, I, S, R, V, Q, A, P, A, ...",2.250074
9777,1.448400,1.018636,1.503468,"[A, Q, S, V, P, W, G, I, S, R, V, Q, A, P, A, ...",2.218206
6360,1.457753,1.015378,1.496100,"[A, Q, S, V, P, W, G, I, S, R, V, Q, A, P, A, ...",2.214481
1546,1.457948,1.018702,1.486360,"[A, Q, S, V, P, A, G, I, S, R, V, Q, A, P, A, ...",2.207564
9844,1.464868,1.017642,1.476096,"[A, Q, S, K, P, W, G, I, S, R, V, Q, A, P, A, ...",2.200432
3049,1.482862,1.142444,1.292327,"[A, Q, S, V, P, W, G, I, S, R, S, Q, A, P, A, ...",2.189315
9696,1.441151,1.012441,1.499173,"[A, Q, S, V, P, D, G, I, S, R, V, Q, A, P, A, ...",2.187414
2909,1.439916,1.030159,1.469270,"[A, Q, S, V, K, W, G, I, S, R, V, Q, A, V, K, ...",2.179429
9335,1.517610,1.167629,1.225401,"[W, Q, S, V, P, W, G, I, S, R, V, Q, A, P, A, ...",2.171416
